In [33]:
%matplotlib notebook

import itertools
import logging
from functools import partial
from collections import defaultdict

import gensim
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors as plt_colors
import numpy as np
import pandas as pnd
import os
from sklearn.cluster import *
from sklearn.preprocessing import normalize
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA, RandomizedPCA
from sklearn.manifold import TSNE
from sklearn import svm, metrics
from multiprocessing import Pool
from knub.thesis.util import *

pnd.set_option("display.max_colwidth", 200)

In [34]:
def evaluate_df(df_param, print_eval):
    for method, df_group in df_param.groupby("method"):
        if "mixture" in method:
            continue
        nr_inclusions = len(df_group)
        nr_succ = len(df_group[df_group.successful])
        if print_eval:
            print "%s: %.2f" % (method, float(nr_succ) / nr_inclusions)
    
def evaluate_file(f, print_eval=True):
    df_file = pnd.read_csv("/home/knub/Repositories/master-thesis/webapp/out/" + f, sep="\t", header=None)
    df_file.columns = ["inclusion_idx", "inclusion_id", "method", "words", "intruder", "selected_word", "successful"]
    evaluate_df(df_file, print_eval)
    return df_file

In [35]:
for f_name in sorted([f for f in os.listdir("../webapp/out") if f.endswith(".txt")]):
    print f_name
    evaluate_file(f_name)
    print

20_stefan.txt
20_lda: 0.65
20_lflda: 0.60
20_topicvec: 0.63
20_welda-gaussian: 0.80

20_tanja.txt
20_lda: 0.70
20_lflda: 0.40
20_topicvec: 0.47
20_welda-gaussian: 0.70

dominik.txt
lda: 0.68
lflda: 0.74
topicvec: 0.49
welda-gaussian: 0.82

joseph.txt
lda: 0.40
lflda: 0.68
topicvec: 0.69
welda-gaussian: 0.60

julian.txt
20_lda: 0.70
20_lflda: 0.45
20_topicvec: 0.74
20_welda-gaussian: 0.65
lflda: 0.44
topicvec: 0.61
welda-gaussian: 0.68

stefan.txt
lda: 0.54
lflda: 0.72
topicvec: 0.65
welda-gaussian: 0.74

tanja.txt
lda: 0.48
lflda: 0.70
topicvec: 0.59
welda-gaussian: 0.66



In [36]:
evaluate_file("julian.txt");

20_lda: 0.70
20_lflda: 0.45
20_topicvec: 0.74
20_welda-gaussian: 0.65
lflda: 0.44
topicvec: 0.61
welda-gaussian: 0.68


In [37]:
evaluate_file("dominik.txt");

lda: 0.68
lflda: 0.74
topicvec: 0.49
welda-gaussian: 0.82


**Evaluating all results**

In [38]:
dfs = []
for f_name in sorted([f for f in os.listdir("../webapp/out") if f.endswith(".txt")]):
    if "mixture" not in f_name:
        dfs.append(evaluate_file(f_name, print_eval=False))

evaluate_df(pnd.concat(dfs), print_eval=True)

20_lda: 0.68
20_lflda: 0.48
20_topicvec: 0.61
20_welda-gaussian: 0.72
lda: 0.53
lflda: 0.66
topicvec: 0.61
welda-gaussian: 0.70
